In [ ]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import torch

curr_directory = os.getcwd()

# Player Roster Information (Copied over from player_roster.ipynb)
teams = ['BOS','BRK','NYK','PHI','TOR','CHI','CLE','DET','IND','MIL','ATL','CHO','MIA','ORL','WAS',
         'DEN','MIN','OKC','POR','UTA','GSW','LAC','LAL','PHO','SAC','DAL','HOU','MEM','NOP','SAS']
        
# Dictionary of roster
# Ex. The roster of Boston Celtics players for the 2019-2020 season can be accessed using roster['BOS']['2019']
# It does not include any players/rookies for which there is no season data
roster = {}
    
for team in teams: 
    roster[team] = {}

# Initialize set for list of all players (with no repeats)
all_players = set()
    
for filename in os.listdir(os.path.join(curr_directory, 'data_sets/player_roster')):
    data = pd.read_csv(os.path.join('data_sets/player_roster', filename))
    year = filename[0:4]
    
    for team in teams:
        roster[team][year] = []
        
        players = data.loc[data['Tm'] == team]
        for ind in players.index: 
            player_name = players['Player'][ind].split('\\', 1)[0]
            if player_name not in roster[team][year]: 
                roster[team][year].append(player_name)
            
        all_players.update(roster[team][year])

num_players = len(all_players)
        
# Player dictionary that maps all players to index
player_index = dict(zip(list(all_players), range(len(all_players))))

game_data = pd.read_csv(os.path.join(curr_directory,'data_sets/nba.games.stats.csv'))

# Sort all values by the Date
game_data = game_data.sort_values(by=['Date'])

game_results = np.array(list(game_data['TeamPoints'] - game_data['OpponentPoints']))
teams = np.array(list(zip(game_data.Team, game_data.Opponent)))
dates = np.array(list(game_data['Date']))

# Remove duplicate games
unique_dates = list(set(dates))
repeat_indexes = []

for date in unique_dates: 
    same_day = np.where(dates == date)
    # suppose same_day = [0, 1, 2, 3, 4, 5]
    for i in same_day[0]: 
        # start with i = 0
        for j in same_day[0]: 
            # j = 0, 1, 2, 3, 4, 5
            if j > i: 
                if np.array_equal(np.flip(teams[j], axis=0) , teams[i]): 
                    repeat_indexes.append(j)

# Final dataset with unique games
unique_game_results = game_results[repeat_indexes]
unique_teams = teams[repeat_indexes]
unique_dates = dates[repeat_indexes]

In [ ]:
class QuadraticRegression:
    def __init__(self, step_size=1e-5, max_iter=1000, eps=1e-3, batch_size =32, theta=None, 
                  verbose=True):
        
        self.theta = theta
        self.batch_size = batch_size
        self.step_size = step_size
        self.max_iter = max_iter
        self.eps = eps
        self.error_list = []
        
    def getSAS(self):
        # Top left
        S = np.array(self.theta[0:962, 0:962])
        # Bottom right
        S2 = np.array(self.theta[963:1925, 963:1925])
        # Top right
        A = np.array(self.theta[0:962, 963:1925])
        # Bottom left
        A2 = np.array(self.theta[963:1925, 0:962])
        
        return S,S2,A,A2

    def predict(self, x): 
        z = x@self.theta@x.T 
        return self.sigmoid(z)
    
    def sigmoid(self, z): 
        return 1.0 / (1. + np.exp(-z))
    
    def loss_function_t(self, theta_t, x, y):
        EPS = 1e-8
        x = torch.tensor(x)
        y = torch.tensor(y)
        p = torch.sigmoid(x @ theta_t @ x.T)
        return -1.*((y * torch.log(p + EPS) + (1-y) * torch.log(1 - p + EPS)).sum())
    
    def pytorch_gradient(self, x, y):
        theta_t = torch.tensor(self.theta, requires_grad=True)
        self.loss_function_t(theta_t, x, y).backward()
        return theta_t.grad.numpy()
    
    def pytorch_batch_gradient(self, x_teams, y_teams, index): 
        x = x_teams[index::self.batch_size]
        y = y_teams[index::self.batch_size]
        
        theta_t = torch.tensor(self.theta, requires_grad=True)
        self.loss_function_t(theta_t, x, y).backward()
        return theta_t.grad.numpy()
    
    def gradBatchLossFunction(self, x_teams, y_teams):
        update = 0
        theta = np.matrix(self.theta)
        
        for i in range(x_teams.shape[0]):
            x = np.matrix(x_teams[i, :])
            y = np.asscalar(y_teams[i])
            update += x.T@x@theta@x.T@x - y*x.T@x
            
        return update
    
    def gradminiBatchLossFunction(self, x_teams, y_teams, batch_size, index):
        update = 0
        theta = np.matrix(self.theta)
        
        for i in range(batch_size):
            x = np.matrix(x_teams[int((i+index) % x_teams.shape[0]), :])
            y = np.asscalar(y_teams[int((i+index) % x_teams.shape[0])])
            update += x.T@x@theta@x.T@x - y*x.T@x
            
        return update
    
    def fit(self, x, y, mini = False):
        iterations = 0
        abs_error = 1
        ind = 0
        
        if self.theta is None: 
            self.theta = np.zeros((2*num_players, 2*num_players))
        
        if mini == False:
            while iterations < self.max_iter and abs_error >= self.eps and abs_error < 1000000:
                error = self.step_size*self.pytorch_gradient(x, y)
                abs_error = np.linalg.norm(error, 2)
                self.error_list.append(abs_error)

                theta_new = self.theta - error
                self.theta = self.project(theta_new)

                iterations += 1

                print('Error {}: {}'.format(iterations, abs_error))
        else:
            batch_num = 1
            while iterations < self.max_iter and abs_error >= self.eps and abs_error < 1000000:
                error = self.step_size*self.pytorch_batch_gradient(x, y, self.batch_size, ind)
                abs_error = np.linalg.norm(error, 2)
                self.error_list.append(abs_error)

                theta_new = self.theta - error
                self.theta = self.project(theta_new)

                iterations += 1
                ind += 1

                print('Error {}: {}'.format(iterations, abs_error))
        
        print('Convergence!')
        plt.plot(self.error_list)
        plt.xlabel('Iterations')
        plt.ylabel('Abs. Error')
        plt.show()
        
    def process_data(self, teams, dates, results): 
        num_games = teams.shape[0]

        # Create x for all games
        # To access x for 0th game -- x[0, :] 
        x_without_intercept = np.zeros((num_games, 2*num_players))
        
        for i in range(num_games): 
            z, t = self.x_for_game(teams[i], dates[i])
            combined = np.vstack((z, t))
            x_without_intercept[i, :] = combined[:, 0]
            
        x = x_without_intercept
        
        # Create y for all games (if team A wins, y = 1; if team B wins, y = 0)
        y = np.zeros((num_games, 1))
        for i in range(num_games): 
            if results[i] > 0: 
                y[i] = 1
            else:
                y[i] = 0
                
        return x, y

    def x_for_game(self, teams, date): 
        x_1 = np.zeros((num_players, 1))
        x_2 = np.zeros((num_players, 1))

        if int(date[5:7]) < 9: 
            year = str(int(date[0:4]) - 1)
        else: 
            year = date[0:4]

        team_1_players = roster[teams[0]][year]
        for item in team_1_players: 
            x_1[player_index[item]] = 1

        team_2_players = roster[teams[1]][year]
        for item in team_2_players: 
            x_2[player_index[item]] = 1

        return x_1, x_2
    
    def add_intercept(self, x): 
        new_x = np.zeros((x.shape[0], x.shape[1] + 1))
        new_x[:, 0] = 1
        new_x[:, 1:] = x
        
        return new_x
    
    def symmetrize(self, m):
        m = np.array(m)
        for i in range(m.shape[0]):
            for j in range(i, m.shape[1]):
                m[i][j] = m[j][i] = 0.5*(m[j][i] + m[i][j])
                
        return m
    
    def antisymmetrize(self, m):
        for i in range(m.shape[0]):
            for j in range(i, m.shape[1]):
                temp = m[i][j] - m[j][i]
                m[i][j] = 0.5*temp
                m[j][i] = -0.5*temp
                
        return m
    
    def project(self, m):
        m = np.array(m)
        side = m.shape[0]
        S = self.symmetrize(m[0:int(side/2 - 1),0:int(side/2 - 1)])
        S_minus = self.symmetrize(m[int(side/2):int(side-1), int(side/2):int(side-1)])

        A = self.antisymmetrize(m[0:int(side/2-1), int(side/2):int(side-1)])
        A_minus = self.antisymmetrize(m[int(side/2):int(side-1), 0:int(side/2-1)])
        S_new = (S - S_minus)/2
        S_minus_new = (S_minus - S)/2
        
        if np.allclose(A, -1*A_minus, 1e-10, 1e-10):
            A_new = A
            A_minus_new = A_minus
        elif np.linalg.norm(A.T - A_minus, 2) < np.linalg.norm(A - A_minus, 2):
            A_new = 0.5*(A + A_minus)
            A_minus_new = A_new.T
        else:
            A_new = 0.5*(A + A_minus.T)
            A_minus_new = A_new.T
            
        M = np.zeros(m.shape)
        M[0:int(side/2 - 1),0:int(side/2 - 1)] = S_new
        M[int(side/2):int(side-1),int(side/2):int(side - 1)] = S_minus_new
        M[0:int(side/2 - 1),int(side/2):int(side - 1)] = A_new
        M[int(side/2):int(side - 1),0:int(side/2 - 1)] = A_minus_new
        
        return M
    
    def general_predict(self, teams, dates, results): 
        test_x, test_y = self.process_data(teams, dates, results)
        
        predicted_y = []
        for i in range(test_x.shape[0]):
            x = test_x[i,:]           
            prediction = self.predict(x)
            if np.asscalar(prediction) > 0.5: 
                predicted_y.append(1)
            else: 
                predicted_y.append(0)

        predicted_y = np.array(predicted_y)
        return np.mean(np.array(predicted_y) == np.array(test_y.T))
    
    def playoff_prediction(self, playoff_filename, playoff_date): 
        # Load playoff data
        playoff_data = pd.read_csv(os.path.join(curr_directory, playoff_filename))

        # Extract features of interest
        raw_playoff_results = np.array(list(playoff_data['PTS'] - playoff_data['PTS.1']))
        raw_playoff_team_pairs = np.array(list(zip(playoff_data['Visitor/Neutral'], playoff_data['Home/Neutral'])))
        raw_playoff_dates = np.array(list(playoff_data['Date']))

        playoff_pairs = {}

        for i in range(len(raw_playoff_team_pairs)): 
            team_1 = raw_playoff_team_pairs[i][0]
            team_2 = raw_playoff_team_pairs[i][1]
            if (team_1,team_2) in playoff_pairs.keys(): 
                # if results > 0 --> team A won --> +1
                # if results < 0 --> team B won --> -1
                if raw_playoff_results[i] > 0: 
                    playoff_pairs[team_1,team_2] += 1
                else: 
                    playoff_pairs[team_1,team_2] += -1
            elif (team_2,team_1) in playoff_pairs.keys():
                # if results > 0 --> team B won --> -1
                # if results < 0 --> team A won --> +1
                if raw_playoff_results[i] > 0: 
                    playoff_pairs[team_2,team_1] += -1
                else: 
                    playoff_pairs[team_2,team_1] += 1
            else: 
                if raw_playoff_results[i] > 0: 
                    playoff_pairs[team_1,team_2] = 1
                else: 
                    playoff_pairs[team_1,team_2] = -1

        playoff_teams = []
        playoff_results = []
        playoff_dates = []

        for key in playoff_pairs: 
            playoff_teams.append([key[0], key[1]])
            playoff_results.append(playoff_pairs[key])
            playoff_dates.append(playoff_date)

        playoff_teams = np.array(playoff_teams)
        playoff_results = np.array(playoff_results)
        playoff_dates = np.array(playoff_dates)

        playoff_x, playoff_y = self.process_data(playoff_teams, playoff_dates, playoff_results)

        predicted_y = []
        for i in range(playoff_x.shape[0]):
            x = playoff_x[i,:]
            prediction = self.predict(x)
            if np.asscalar(prediction) > 0.5: 
                predicted_y.append(1)
            else: 
                predicted_y.append(0)

        predicted_y = np.array(predicted_y)
        prediction_accuracy = np.mean(np.array(predicted_y) == np.array(playoff_y.T[0][:]))

        return prediction_accuracy

In [ ]:
def learning_rate_tune(teams, dates, results, playoff_filename, date): 
    lr = [1e-3, 1e-4, 1e-5]
    errors = {}
    dev_accuracies = []
    playoff_accuracies = []
    n = teams.shape[0]
    train_n = int(np.floor(0.8*n))
    
    for i in lr: 
        print('Learning rate: {}'.format(i))
        # Initialize the model and process the data
        model = QuadraticRegression(step_size=i)
        teams_s, dates_s, results_s = shuffle(teams, dates, results, random_state=0)
        x, y = model.process_data(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
        # Fit the model (w/o mini-batch) on training set
        model.fit(x, y, mini=False)
        errors[str(i)] = model.error_list
        
        dev_accuracy = model.general_predict(teams_s[train_n+1:n], dates_s[train_n+1:n], results_s[train_n+1:n])
        dev_accuracies.append(dev_accuracy)
        
        playoff_accuracy = model.playoff_prediction(playoff_filename, date)
        playoff_accuracies.append(playoff_accuracy)
    
    return errors, dev_accuracies, playoff_accuracies

def learning_rate_tune_all(teams, dates, results): 
    lr = [1e-3, 1e-4, 1e-5]
    errors = {}
    dev_accuracies = []
    n = teams.shape[0]
    train_n = int(np.floor(0.8*n))
    
    for i in lr: 
        print('Learning rate: {}'.format(i))
        # Initialize the model and process the data
        model = QuadraticRegression(step_size=i, max_iters=1000)
        teams_s, dates_s, results_s = shuffle(teams, dates, results, random_state=0)
        x, y = model.process_data(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
        # Fit the model (w/o mini-batch) on training set
        start = time.time()
        model.fit(x, y, mini=False)
        
        errors[str(i)] = model.error_list
        
        dev_accuracy = model.general_predict(teams_s[train_n+1:n], dates[train_n+1:n], results_s[train_n+1:n])
        dev_accuracies.append(dev_accuracy)
    
    return errors, dev_accuracies

In [ ]:
# Final Experiments

# Full dataset
teams_s, dates_s, results_s = shuffle(unique_teams, unique_dates, unique_game_results, random_state=0)
train_n = 3935
full_n = 4919

# Runs w/o mini-batch for different learning rates (lr = 1E-6, 5E-7, 1E-7)
print('Starting full batch descent with lr = 1E-6')
model_lr_1e6 = QuadraticRegression(step_size = 1e-6, max_iter = 500)
x_lr_1e6, y_lr_1e6= model_lr_1e6.process_data(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
start = time.time()
model_lr_1e6.fit(x_lr_1e6, y_lr_1e6, mini=False)
end = time.time()
print('Time: {}'.format(end-start))
train_accuracy_1e6 = model_lr_1e6.general_predict(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
dev_accuracy_1e6 = model_lr_1e6.general_predict(teams_s[train_n+1:full_n], dates_s[train_n+1:full_n], 
                                                results_s[train_n+1:full_n])
errors_lr_1e6 = model_lr_1e6.error_list
print('Train Accuracy: {}'.format(train_accuracy_1e6))
print('Dev Accuracy: {}'.format(dev_accuracy_1e6))

In [ ]:
# Max_iters to be adjusted based on results above.
print('Starting full batch descent with lr = 5E-7')
model_lr_5e7 = QuadraticRegression(step_size = 5e-6, max_iter = 500)
x_lr_5e7, y_lr_5e7= model_lr_5e7.process_data(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
start = time.time()
model_lr_5e7.fit(x_lr_5e7, y_lr_5e7, mini=False)
end = time.time()
print('Time: {}'.format(end-start))
train_accuracy_5e7 = model_lr_5e7.general_predict(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
dev_accuracy_5e7 = model_lr_5e7.general_predict(teams_s[train_n+1:full_n], dates_s[train_n+1:full_n], 
                                                results_s[train_n+1:full_n])
errors_lr_5e7 = model_lr_5e7.error_list
print('Train Accuracy: {}'.format(train_accuracy_5e7))
print('Dev Accuracy: {}'.format(dev_accuracy_5e7))

print('Starting full batch descent with lr = 1E-7')
model_lr_1e7 = QuadraticRegression(step_size = 1e-6, max_iter = 500)
x_lr_1e7, y_lr_1e7= model_lr_1e7.process_data(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
start = time.time()
model_lr_1e7.fit(x_lr_1e7, y_lr_1e7, mini=False)
end = time.time()
print('Time: {}'.format(end-start))
train_accuracy_1e7 = model_lr_1e7.general_predict(teams_s[0:train_n], dates_s[0:train_n], results_s[0:train_n])
dev_accuracy_1e7 = model_lr_1e7.general_predict(teams_s[train_n+1:full_n], dates_s[train_n+1:full_n], 
                                                results_s[train_n+1:full_n])
errors_lr_1e7 = model_lr_1e7.error_list
print('Train Accuracy: {}'.format(train_accuracy_1e7))
print('Dev Accuracy: {}'.format(dev_accuracy_1e7))

In [ ]:
print('Season: 2014-2015')
errors_2015, dev_2015, playoff_2015 = learning_rate_tune(unique_teams[0:1229], unique_dates[0:1229], 
                                                         unique_game_results[0:1229], 'data_sets/2015_playoffs.csv', '2015-04-10')
print('Season: 2015-2016')
errors_2016, dev_2016, playoff_2016 = learning_rate_tune(unique_teams[1230:2459], unique_dates[1230:2459], 
                                                         unique_game_results[1230:2459], 'data_sets/2016_playoffs.csv', '2016-04-10')
print('Season: 2016-2017')
errors_2017, dev_2017, playoff_2017 = learning_rate_tune(unique_teams[2460:3689], unique_dates[2460:3689], 
                                                         unique_game_results[2460:3689], 'data_sets/2017_playoffs.csv', '2017-04-10')
print('Season: 2017-2018')
errors_2018, dev_2018, playoff_2018 = learning_rate_tune(unique_teams[3690:4919], unique_dates[3690:4919], 
                                                         unique_game_results[3690:4919], 'data_sets/2018_playoffs.csv', '2018-04-10')